In [219]:
import numpy as np
import seaborn as sns
import pandas as pd
import sys
import matplotlib.pyplot as plt
from sklearn import datasets, model_selection
sns.set(rc={'figure.figsize':(10,10)})

Leo Brieman [first described bagging in 1996](https://link.springer.com/content/pdf/10.1023%2FA%3A1018054314350.pdf)

### Get some data

In [281]:
boston_data = datasets.load_boston()
df_boston = pd.DataFrame(boston_data.data,columns=boston_data.feature_names)
df_boston['target'] = pd.Series(boston_data.target)
train, test = model_selection.train_test_split(df_boston, test_size=0.2)
train_X = train.drop('target', axis=1).values
train_y = train['target'].values
test_X = test.drop('target', axis=1).values
test_y = test['target'].values
train.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
76,0.10153,0.0,12.83,0.0,0.437,6.279,74.5,4.0522,5.0,398.0,18.7,373.66,11.97,20.0
409,14.43830,0.0,18.10,0.0,0.597,6.852,100.0,1.4655,24.0,666.0,20.2,179.36,19.78,27.5
214,0.28955,0.0,10.59,0.0,0.489,5.412,9.8,3.5875,4.0,277.0,18.6,348.93,29.55,23.7
13,0.62976,0.0,8.14,0.0,0.538,5.949,61.8,4.7075,4.0,307.0,21.0,396.90,8.26,20.4
145,2.37934,0.0,19.58,0.0,0.871,6.130,100.0,1.4191,5.0,403.0,14.7,172.91,27.80,13.8


### Regression Tree Algorithm

In [343]:
def compute_sum_split_variance(xs, y, v):
    '''xs - 1D array of scalars
        v - scalar to split on'''
    left = y[xs <= v]
    right = y[xs > v]
    left_var = 0 if len(left) == 0 else ((left - left.mean()) ** 2).sum()
    right_var = 0 if len(right) == 0 else ((right - right.mean()) ** 2).sum()
    return  left_var + right_var

def node(i, s, p, c, l, r):
    return {'internal': i,
            'split': s,
            'p': p,
            'c':c,
            'l':l,
            'r':r}

def splitf(X, y, max_leaf_n):
    if X.shape[0] <= max_leaf_n:
        return node(False, None, None, y.mean(), None, None)
    
    lowest_var, best_p_idx, best_split = sys.float_info.max, None, None
    for p_idx in range(0, X.shape[1]):
        for n_idx in range(0, X.shape[0]):
            split = X[n_idx][p_idx]
            var = compute_sum_split_variance(X[:,p_idx], y, split)
            if var < lowest_var:
                lowest_var = var
                best_p_idx = p_idx
                best_split = split
    left_idxs = X[:, best_p_idx] <= best_split
    right_idxs = X[:, best_p_idx] > best_split
    
    if len(y[left_idxs]) == 0 or len(y[right_idxs]) == 0:
        # this can only happen if yi = yi+1 ... = yn
        # in which case.. there is no benefit to branching more.
        assert(np.unique(y).size == 1)
        return node(False, None, None, y.mean(), None, None)
    
    l = splitf(X[left_idxs], y[left_idxs], max_leaf_n)
    r = splitf(X[right_idxs], y[right_idxs], max_leaf_n)
    return node(True, best_split, best_p_idx, None, l, r)

def predict(x, model):
    if not model['internal']:
        return model['c']
    if x[model['p']] <= model['split']:
        return predict(x, model['l'])
    else:
        return predict(x, model['r'])

In [344]:
max_leaf_n = 2
tree = splitf(train_X, train_y, max_leaf_n)

# evaluate training error
preds = [predict(r, tree) for r in train_X]
print('training error: {}'.format(np.sqrt(((train_y - preds) ** 2).sum() / train_y.shape[0])))

# evaluate test error
preds = [predict(r, tree) for r in test_X]
print('test error: {}'.format(np.sqrt(((test_y - preds) ** 2).sum() / test_y.shape[0])))

training error: 0.4043543931349389
test error: 4.514991368632229


### Bagging

In [345]:
def bagged_predict(x, trees):
    return np.array([predict(x, t) for t in trees]).mean()
    
def bagged_trees(X, y, max_leaf_n, b):
    n = X.shape[0]
    trees = []
    for _ in range(0,b):
        b_sample = np.random.randint(0, n, n)
        trees.append(splitf(X[b_sample], y[b_sample], max_leaf_n))
    return trees

In [353]:
max_leaf_n = 2
b = 50
trees = bagged_trees(train_X, train_y, max_leaf_n, b)

# evaluate training error
preds = [bagged_predict(x, trees) for x in train_X]
print('training error: {}'.format(np.sqrt(((train_y - preds) ** 2).sum() / train_y.shape[0])))

# evaluate test error
preds = [bagged_predict(x, trees) for x in test_X]
print('test error: {}'.format(np.sqrt(((test_y - preds) ** 2).sum() / test_y.shape[0])))

training error: 1.3357320034904139
test error: 3.5211720697940962
